In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import keras_spiking

tf.random.set_seed(0)
np.random.seed(0)

In [10]:
# Load MNIST Digits Dataset
((train_images, train_labels),(test_images, test_labels),) = tf.keras.datasets.mnist.load_data()
#train_labels = train_labels.squeeze()
#test_labels = test_labels.squeeze()

# maximum of each colour is 255
#print(np.max(train_images[0], axis=(0,1)))

# normalize images so values are between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

#class_names = [
#    "airplane",
#    "automobile",
#    "bird",
#    "cat",
#    "deer",
#    "dog",
#    "frog",
#    "horse",
#    "ship",
#    "truck",
#]

class_names = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]

num_classes = len(class_names)


In [11]:
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

In [12]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
        
        #tf.keras.layers.Flatten(input_shape=train_images.shape[1:]),
        #tf.keras.layers.Dense(128, activation="relu"),
        #tf.keras.layers.Dense(num_classes),
    ]
)


def train(input_model, train_x, test_x):
    input_model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    input_model.fit(train_x, train_labels, epochs=10)

    _, test_acc = input_model.evaluate(test_x, test_labels, verbose=2)


    print("\nTest accuracy:", test_acc)


train(model, train_images, test_images)

Epoch 1/10
1875/1875 [==============================] - 22s 11ms/step - loss: 0.3507 - accuracy: 0.8939
Epoch 2/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0440 - accuracy: 0.9867
Epoch 3/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0321 - accuracy: 0.9896
Epoch 4/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0234 - accuracy: 0.9921
Epoch 5/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0176 - accuracy: 0.9946
Epoch 6/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0138 - accuracy: 0.9952
Epoch 7/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0127 - accuracy: 0.9955
Epoch 8/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0094 - accuracy: 0.9969
Epoch 9/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0095 - accuracy: 0.9970
Epoch 10/10
1875/1875 [==============================] - 22s 12m

In [13]:
# repeat the images for n_steps
n_steps = 10
train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1, 1))
test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1, 1))

In [ ]:
spikeaware_model = tf.keras.Sequential(
    [
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(28, 28, 1))),
        keras_spiking.SpikingActivation("relu", dt=0.01, spiking_aware_training=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(64, (3, 3))),
        keras_spiking.SpikingActivation("relu", dt=0.01, spiking_aware_training=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(64, (3, 3))),
        
        tf.keras.layers.Reshape((-1, 3 * 3 * 64), input_shape=(None,10,3,3,64)),
        keras_spiking.SpikingActivation("relu", dt=0.01, spiking_aware_training=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64)),
        # set spiking_aware training and a moderate dt
        keras_spiking.SpikingActivation("relu", dt=0.01, spiking_aware_training=True),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(10),
    ]
)

# train the model, identically to the non-spiking version,
# except using the time sequences as inputs
train(spikeaware_model, train_sequences, test_sequences)

Epoch 1/10
1875/1875 [==============================] - 518s 275ms/step - loss: 3.9617 - accuracy: 0.5447
Epoch 2/10
1875/1875 [==============================] - 525s 280ms/step - loss: 0.3393 - accuracy: 0.9038
Epoch 3/10
1875/1875 [==============================] - 562s 300ms/step - loss: 0.2054 - accuracy: 0.9387
Epoch 4/10
 475/1875 [======>.......................] - ETA: 6:51 - loss: 0.1522 - accuracy: 0.9538